In [6]:
import pandas as pd
import os


In [131]:
# This code reads in a list of CSV files from the "./dataset" directory
# and processes them by cleaning up any string columns, converting the
# timestamp column to a numerical format, and saving the cleaned-up
# data to the "./processed" directory

datasets = os.listdir("./dataset")

for dataset in datasets:
    # Read in the CSV file
    df = pd.read_csv(f"./dataset/{dataset}", header=None, delimiter=" ")

    # Clean up any string columns by removing '[' and ']' characters
    for col in df.columns:
        if df[col].dtype == 'object':  # Check if column contains strings
            df[col] = df[col].str.replace(']', '')
            df[col] = df[col].str.replace('[', '')
        elif df[col].dtype == 'int64':  # Check if column contains integers
            raise ValueError("Cannot remove ']' from columns of integer type")

    # Drop the first column, which is not needed
    df = df.drop(columns=[0])

    # Rename the second and third columns to "timestamp" and "Value"
    df = df.rename(columns={1: 'timestamp', 2: 'Value'})

    # Convert the "timestamp" column to a timedelta format
    df['timestamp'] = pd.to_timedelta(df['timestamp'])

    # Convert the timedelta to seconds
    df['timestamp'] = (df['timestamp'] - df['timestamp'].iloc[0]).dt.total_seconds()

    # Remove any rows where the timestamp is equal to 0
    df = df[df['timestamp']//1 != 0]

    # Round the timestamp to the nearest 0.001 seconds
    df['timestamp'] = round(df["timestamp"]-1, 3)

    # Remove any duplicate rows
    # df = df.drop_duplicates(subset=['timestamp'])

    # Save the cleaned-up data to the "./processed" directory
    df.to_csv(f"./processed/{dataset}", index=False)

In [132]:
df = pd.read_csv("./processed/backwardb.csv")

index_to_drop = df.index[df['timestamp'] == 57.003][0]

# Drop values until the index where 57.029 is encountered
df = df.drop(range(index_to_drop))
df['timestamp'] = (df['timestamp'] - df['timestamp'].iloc[0])

In [133]:
df.to_csv(f"./processed/backwardb.csv", index=False, float_format="%.3f")